<a href="https://colab.research.google.com/github/rboghe/cened/blob/master/cened_1_2_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [106]:
import os
import urllib.request
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, make_scorer
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as stats
from lightgbm import LGBMRegressor

# Silence panda's setting with copy warning
pd.options.mode.chained_assignment = None

# Load data

In [107]:
# Cened 1.2
url = 'https://www.dati.lombardia.it/api/views/rsg3-xhvk/rows.csv?accessType=DOWNLOAD'


if os.path.isfile('/tmp/cened12.csv'):
  pass
else:
  urllib.request.urlretrieve(url, '/tmp/cened12.csv')

In [108]:
# DDH
url = 'https://raw.githubusercontent.com/rboghe/cened/master/degreedays.txt'

urllib.request.urlretrieve(url, '/tmp/ddh.csv')

ddh = pd.read_csv('/tmp/ddh.csv', usecols = ['comune','dd'])

In [109]:
cols = ['COMUNE','FOGLIO', 'PARTICELLA','SUPERFICIE_DISPERDENTE',
          'TRASMITTANZA_MEDIA_COPERTURA','TRASMITTANZA_MEDIA_INVOLUCRO',
          'TRASMITTANZA_MEDIA_BASAMENTO','TRASMITTANZA_MEDIA_SERRAMENTO',
          'SUPERFICIE_LORDA', 'VOLUME_LORDO', 'DESTINAZIONE_DI_USO',
          'ANNO_COSTRUZIONE','SUPERFICIE_VETRATA_OPACA', 'VOLUME_NETTO',
          'SUPERFICIE_NETTA', 'EPH']
  
cened_old = pd.read_csv('/tmp/cened12.csv', usecols = cols)

# Preprocessing

In [110]:
# Drop buildings with NaN values
cened_old.dropna(inplace = True)

# We'll use residential buildings only
cened_old = cened_old[cened_old['DESTINAZIONE_DI_USO'] == 'E.1(1)']

# Add ddh
cened_old['COMUNE'] = cened_old['COMUNE'].str.lower()
ddh['comune'] = ddh['comune'].str.lower()
cened_old['COMUNE'] = cened_old['COMUNE'].str.replace(r"o`",  "o'")
cened_old['COMUNE'] = cened_old['COMUNE'].str.replace(r"baranzate",  "bollate")
cened_old['COMUNE'] = cened_old['COMUNE'].str.replace(r"bovisio masciago",  "bovisio-masciago")
cened_old['COMUNE'] = cened_old['COMUNE'].str.replace(r"cornate d`adda",  "cornate d'adda")
cened_old['COMUNE'] = cened_old['COMUNE'].str.replace(r"cortenuova",  "cortenova")
cened_old['COMUNE'] = cened_old['COMUNE'].str.replace(r"sant'omobono terme",  "sant'omobono imagna")
ddh['comune'] = ddh['comune'].str.replace(r"è",  "e'")
ddh['comune'] = ddh['comune'].str.replace(r"è",  "e'")
ddh['comune'] = ddh['comune'].str.replace(r"é",  "e'")
ddh['comune'] = ddh['comune'].str.replace(r"ò",  "o'")
ddh['comune'] = ddh['comune'].str.replace(r"ù",  "u'")

cened_old = cened_old.merge(ddh, left_on='COMUNE', right_on='comune', how='left')



# Correct floats
for col in ['SUPERFICIE_LORDA', 'SUPERFICIE_NETTA', 'VOLUME_LORDO', 'VOLUME_NETTO',
       'SUPERFICIE_DISPERDENTE', 'SUPERFICIE_VETRATA_OPACA',
       'TRASMITTANZA_MEDIA_INVOLUCRO', 'TRASMITTANZA_MEDIA_COPERTURA',
       'TRASMITTANZA_MEDIA_BASAMENTO', 'TRASMITTANZA_MEDIA_SERRAMENTO', 'EPH', 'dd']:
    cened_old[col] = cened_old[col].astype(str)
    cened_old[col] = cened_old[col].str.replace(r',', '')
    cened_old[col] = cened_old[col].astype("float")

# Filtering

In [111]:
# Drop buildings with wrong thermal conductivity
for col in ['TRASMITTANZA_MEDIA_INVOLUCRO', 'TRASMITTANZA_MEDIA_COPERTURA',
       'TRASMITTANZA_MEDIA_BASAMENTO']:
    cened_old = cened_old[cened_old[col] > 0]
    cened_old = cened_old[cened_old[col] < 4]

cened_old = cened_old[cened_old['TRASMITTANZA_MEDIA_SERRAMENTO'] > 0]
cened_old = cened_old[cened_old['TRASMITTANZA_MEDIA_SERRAMENTO'] < 6]

# Drop buildings with wrong EPH
cened_old = cened_old[cened_old['EPH'] > 0]
cened_old = cened_old[cened_old['EPH'] < 1000]

# Drop buildings with wrong An
cened_old = cened_old[cened_old['SUPERFICIE_NETTA'] > 50]

# Drop buildings with wrong Vn
cened_old = cened_old[cened_old['VOLUME_NETTO'] > 150]

# Drop buildings with wrong average height
cened_old['ALTEZZA_MEDIA'] = cened_old['VOLUME_NETTO']/cened_old['SUPERFICIE_NETTA']
cened_old = cened_old[cened_old['ALTEZZA_MEDIA'] > 2.4]


# Drop buildings with wrong mean thermal conductivity
cened_old['TRASMITTANZA_MEDIA'] = (cened_old['TRASMITTANZA_MEDIA_SERRAMENTO'] + cened_old['TRASMITTANZA_MEDIA_COPERTURA'] +
         cened_old['TRASMITTANZA_MEDIA_BASAMENTO'] + cened_old['TRASMITTANZA_MEDIA_INVOLUCRO'])/4
cened_old = cened_old[cened_old['TRASMITTANZA_MEDIA'] > 0.15]
cened_old = cened_old[cened_old['TRASMITTANZA_MEDIA'] < 4]

# Feature engineering

In [112]:
# Opaque surface
cened_old['SUP_OPACA'] = cened_old['SUPERFICIE_DISPERDENTE']/(1+cened_old['SUPERFICIE_DISPERDENTE'])

# Opaque surface
cened_old['SUP_FINESTRATA'] = cened_old['SUPERFICIE_DISPERDENTE'] - cened_old['SUP_OPACA']

# Take care of construction year
cened_old.ANNO_COSTRUZIONE = cened_old.ANNO_COSTRUZIONE.astype(str)
cened_old.ANNO_COSTRUZIONE = cened_old.ANNO_COSTRUZIONE.map(lambda x:x[-4:])
cened_old.ANNO_COSTRUZIONE = cened_old.ANNO_COSTRUZIONE.astype(int)

# Shuffle

In [113]:
cened = cened_old.sample(frac=1).reset_index(drop=True)

# Reserve a test set

In [114]:
msk = np.random.rand(len(cened)) < 0.8
train = cened[msk]
test = cened[~msk]

In [115]:
print(len(train))

210751


In [116]:
print(len(test))

52799


# Define MAPE

In [117]:
def neg_mape(y_true, y_pred):
  y_true = np.array(y_true)
  y_pred = np.array(y_pred)
  return -np.abs((y_true - y_pred)/y_true).mean()

In [118]:
def modified_neg_mape(y_true, y_pred):
  y_true = np.array(y_true)
  y_pred = np.array(y_pred)
  errors = np.abs((y_true - y_pred)/y_true)
  index = np.argwhere(errors > 1)
  mod_errors = np.delete(errors, index)
  return -mod_errors.mean()

In [119]:
neg_mape_scorer = make_scorer(neg_mape, greater_is_better=True)

In [120]:
modified_neg_mape_scorer = make_scorer(modified_neg_mape, greater_is_better=True)

# Random search

In [121]:
mlcol = ['ANNO_COSTRUZIONE','SUPERFICIE_LORDA', 'SUPERFICIE_NETTA', 'VOLUME_LORDO', 
         'VOLUME_NETTO', 'SUPERFICIE_DISPERDENTE',
         'TRASMITTANZA_MEDIA_INVOLUCRO', 'TRASMITTANZA_MEDIA_COPERTURA',
         'TRASMITTANZA_MEDIA_BASAMENTO', 'TRASMITTANZA_MEDIA_SERRAMENTO',
         'dd','SUP_FINESTRATA','SUP_OPACA']

In [122]:
param_dist = {'feature_fraction': np.linspace(0.4, 1, num=7),
              'num_leaf' : list(range(20,40)),
             'max_depth' : [-1],
             'max_bin' : [100, 200, 300, 500, 750, 1000, 2000],
             'bagging_fraction' : np.linspace(0.4, 1, num=14),
             'bagging_freq' : list(range(1,10)),
             'lambda_l1' : stats.uniform(0, 0.6),
             'lambda_l2' : stats.uniform(0, 0.6)}

lgbm = LGBMRegressor(n_estimators = 100, silent = True, verbose = 0, is_training_metric = True, n_jobs = 1, 
                     eval_metric  = 'mape')

n_iter_search = 100

random_search = RandomizedSearchCV(lgbm, param_distributions=param_dist, n_iter=n_iter_search, 
                                   scoring=modified_neg_mape_scorer, cv = 5, n_jobs = 6, verbose = 2)

random_search.fit(cened[mlcol], cened['EPH'])

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:  1.6min
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:  8.0min
[Parallel(n_jobs=6)]: Done 353 tasks      | elapsed: 18.0min
[Parallel(n_jobs=6)]: Done 500 out of 500 | elapsed: 24.6min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=LGBMRegressor(boosting_type='gbdt',
                                           class_weight=None,
                                           colsample_bytree=1.0,
                                           eval_metric='mape',
                                           importance_type='split',
                                           is_training_metric=True,
                                           learning_rate=0.1, max_depth=-1,
                                           min_child_samples=20,
                                           min_child_weight=0.001,
                                           min_split_gain=0.0, n_estimators=100,
                                           n_jobs=1, num_leaves=31,
                                           objective=None, random_sta...
                                        'lambda_l1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fd007440908>,
            

# Print results

In [123]:
print(random_search.best_params_)

{'bagging_fraction': 0.9538461538461538, 'bagging_freq': 4, 'feature_fraction': 0.5, 'lambda_l1': 0.12344304587385713, 'lambda_l2': 0.14934942051074568, 'max_bin': 1000, 'max_depth': -1, 'num_leaf': 31}


In [124]:
print(random_search.best_score_)

-0.17476686516762513


In [135]:
# Create a results df
cv_df = pd.DataFrame(random_search.cv_results_)
cv_df = cv_df.sort_values(by = ['rank_test_score']).reset_index()

# Filter columns
res_cols = [col for col in cv_df if col.startswith('split')]

# Select best iter
best_res = cv_df.loc[0]

# Print results for each fold
print(best_res[res_cols])

split0_test_score   -0.174701
split1_test_score   -0.175215
split2_test_score    -0.17495
split3_test_score   -0.174548
split4_test_score   -0.174421
Name: 0, dtype: object


# Train the final model

In [125]:
light = LGBMRegressor(n_estimators = 1000, silent = False, verbose = 2, is_training_metric = True, n_jobs = 6)
light.set_params(**random_search.best_params_)

LGBMRegressor(bagging_fraction=0.9538461538461538, bagging_freq=4,
              boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              feature_fraction=0.5, importance_type='split',
              is_training_metric=True, lambda_l1=0.12344304587385713,
              lambda_l2=0.14934942051074568, learning_rate=0.1, max_bin=1000,
              max_depth=-1, min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=1000, n_jobs=6, num_leaf=31,
              num_leaves=31, objective=None, random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=False, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0, verbose=2)

In [126]:
light.fit(train[mlcol], train['EPH'])

LGBMRegressor(bagging_fraction=0.9538461538461538, bagging_freq=4,
              boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              feature_fraction=0.5, importance_type='split',
              is_training_metric=True, lambda_l1=0.12344304587385713,
              lambda_l2=0.14934942051074568, learning_rate=0.1, max_bin=1000,
              max_depth=-1, min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=1000, n_jobs=6, num_leaf=31,
              num_leaves=31, objective=None, random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=False, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0, verbose=2)

# Test the model

In [127]:
y_pred = light.predict(test[mlcol])

In [128]:
results_df = pd.DataFrame({'predicted' : y_pred, 'true' : test['EPH']})
results_df['error'] = np.abs((results_df['true'] - results_df['predicted'])/results_df['true'])*100

In [131]:
# MAPE
results_df['error'].mean()

20.879417921160034

In [130]:
# MAPE without >100%
results_df[results_df.error <= 100].error.mean()

16.612100347667194

In [132]:
results_df

,predicted,true,error
6,188.723521,203.7737,7.385732
12,351.597201,457.9454,23.222899
13,47.959828,48.3549,0.817025
14,281.416392,253.8359,10.865481
16,276.038918,244.0243,13.119439
...,...,...,...
263526,159.051829,125.6282,26.605196
263532,202.756679,170.7102,18.772445
263545,251.084710,269.6724,6.892693
263546,244.227630,159.6327,52.993485
